In [16]:
import random
import re
import secrets
import pandas as pd
import numpy as np
from flask import Flask, render_template, flash, redirect, url_for, session, request, jsonify

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from joblib import load

In [5]:
pip install flask_sqlalchemy


Defaulting to user installation because normal site-packages is not writeable
  Using cached flask_sqlalchemy-3.1.1-py3-none-any.whl.metadata (3.4 kB)
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
    --------------------------------------- 0.0/2.1 MB 660.6 kB/s eta 0:00:04
   -- ------------------------------------- 0.1/2.1 MB 1.8 MB/s eta 0:00:02
   ------ --------------------------------- 0.3/2.1 MB 3.0 MB/s eta 0:00:01
   ------------- -------------------------- 0.7/2.1 MB 4.3 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.1 MB 4.8 MB/s eta 0:00:01
   ------------------------- -------------- 1.4/2.1 MB 5.4 MB/s eta 0:00:01
   --------------------------------- ------ 1.7/2.1 MB 5.8 MB/s eta 0:00:01
   ---------------------------------------  2.1/2.1 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 5.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/299.7 kB ? eta -:--:--
   --------------------


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
from flask_sqlalchemy import SQLAlchemy


In [18]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'
app.secret_key = "m4xpl0it"
db = SQLAlchemy(app)

In [19]:
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)
    password = db.Column(db.String(80), nullable=False)


In [20]:
# Helper Functions
def make_token():
    """Creates a cryptographically-secure, URL-safe string"""
    return secrets.token_urlsafe(16)


In [21]:
# Routes
@app.route("/")
def index():
    return render_template("index.html") 

    

In [24]:
@app.route("/login", methods=["GET", "POST"])

def login():
    if request.method == "POST":
        uname = request.form.get("uname")
        passw = request.form.get("passw")
        user = User.query.filter_by(username=uname, password=passw).first()
        if user:
            session['user_id'] = user.id
            return redirect(url_for("index_auth"))
        else:
            flash("Invalid username or password")
    return render_template("login.html")



In [25]:
@app.route("/register", methods=["GET", "POST"])
def register():
    if request.method == "POST":
        uname = request.form['uname']
        mail = request.form['mail']
        passw = request.form['passw']
        new_user = User(username=uname, email=mail, password=passw)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for("login"))
    return render_template("register.html")

In [26]:
def predict_disease_from_symptom(symptom_list):
    try:
        # Load pre-trained model
        clf = load("path/to/random_forest.joblib")
        symptoms = {s: 0 for s in symptom_list}
        
        # Set symptoms based on input
        for symptom in symptom_list:
            if symptom in symptoms:
                symptoms[symptom] = 1
        
        # Create DataFrame for model prediction
        df_test = pd.DataFrame([symptoms])
        result = clf.predict(df_test)[0]
        return f"Disease: {result}"
    except Exception as e:
        print(f"Prediction error: {e}")
        return "Prediction failed. Please try again."

In [ ]:
# Disease Prediction Function


# Chat Message Route
@app.route('/ask', methods=['GET', 'POST'])
def chat_msg():
    user_message = request.args.get("message", "").lower()
    session_id = request.args.get("sessionId")

In [ ]:
if user_message == "undefined":
        response = ["Welcome! Please enter your name to start."]
else:
        response = handle_chatbot_response(user_message, session_id)

    return jsonify({'status': 'OK', 'answer': response})



In [ ]:

 # Chatbot Logic   
# Running the Flask app
if __name__ == "__main__":
    db.create_all()
    app.run(debug=True)
